<a href="https://colab.research.google.com/github/varshaelza/Text-Summarization-Categorization/blob/main/CNNBasedSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentence to Salience

In [ ]:
import pandas as pd
t = pd.read_csv('/content/drive/MyDrive/BBC Dataset/LDA Summarization/LDASummariestrain.csv') 

In [ ]:
t

In [ ]:
rawData = {}
rawSummaries = {}
    
# running count variable -- keeps track of the total size
totalSentences = 0 

In [ ]:
def sep(text):
  import nltk
  nltk.download('punkt')
  import re
  text= text.replace("\t"," ")
  text=re.sub(r'\.+', '.',text)
  k=0
  c=0
  p=0
  a_list=[]
  while k<len(text):
    if c==0 and text[k]=='.':
      a_list.append(text[p:k+1])
      p=k+1
    elif c==0 and text[k]=='"':
      c=1
    elif c==1 and text[k]=='"' and text[k-1]=="." :
      a_list.append(text[p:k+1])
      p=k+1
      c=0
    elif c==1 and text[k]=='"' and text[k-2]=="." :
      a_list.append(text[p:k+1])
      p=k+1
      c=0
    elif c==1 and text[k]=='"':
      c=0
    k=k+1
  if p<len(text)-1:
    a_list.append(text[p:len(text)])
  return a_list
  

In [ ]:
def spl(text):
  t=text.split('\n',1)
  if len(t)!=2:
    return ""
  else :
    return t[1]

In [ ]:
j=0
while j<len(t):
  a=sep(t['News'][j])
  a[0]=spl(a[0])
  a[0]=spl(a[0])
  if a[0]=="":
    del a[0]
  i=0
  while i<len(a):
    a[i] = a[i].replace('\n',' ')
    a[i] = a[i].replace('\t',' ')
    a[i]= a[i].replace(r'\.+', ".")
    a[i]= a[i].replace(r'\ +', " ")
    i=i+1

  b=sep(t['Summary'][j])
  b[0]=spl(b[0])
  b[0]=spl(b[0])
  if b[0]=="":
    del b[0]
  i=0
  while i<len(b):
    b[i] = b[i].replace('\n',' ')
    b[i] = b[i].replace('\t',' ')
    b[i]= b[i].replace(r'\.+', ".")
    b[i]= b[i].replace(r'\ +', " ")
    i=i+1
  
  rawData[j]=a
  rawSummaries[j]=b
  j=j+1


In [ ]:
!pip install rouge/requirements.txt
!pip install rouge-score
import numpy as np

def calc_rouge_scores(pred_summaries, gold_summaries, 
                                 keys=['rouge1', 'rouge2'], use_stemmer=True):
    #Calculate rouge scores
    from rouge_score import rouge_scorer
    scorer = rouge_scorer.RougeScorer(keys, use_stemmer= use_stemmer)
    n = len(pred_summaries)
    scores = [scorer.score(pred_summaries[j], gold_summaries[j]) for 
              j in range(n)] 
              
    #create dict
    dict_scores={}                                                            
    for key in keys:
        dict_scores.update({key: {}})
        
    #populate dict    
    for key in keys:
        
        precision_list = [scores[j][key][0] for j in range(len(scores))]
        recall_list = [scores[j][key][1] for j in range(len(scores))]
        f1_list = [scores[j][key][2] for j in range(len(scores))]

        precision = np.mean(precision_list)
        recall = np.mean(recall_list)
        f1 = np.mean(f1_list)
        
        dict_results = {'recall': recall, 'precision': precision, 'f1': f1}
        
        dict_scores[key] = dict_results
        
    return dict_scores

In [ ]:
saliency={}
i=0
while i<len(rawData):
  saliency[i]=[]
  j=0
  while j<len(rawData[i]):
    s=calc_rouge_scores([rawData[i][j]],[t['Summary'][i]])
    saliency[i].append(0.5*s['rouge1']['f1']+s['rouge2']['f1']*0.5) #alpha=0.5
    j=j+1
  i=i+1

In [ ]:
tot=0;
i=0
while i<len(rawData):
  tot=tot+len(rawData[i])
  i=i+1

In [ ]:
import numpy as np
nx3output = np.zeros((tot, 3), dtype=object)

In [ ]:
i=0
c=0
while i<len(rawData):
  j=0
  while j<len(rawData[i]):
    nx3output[c,0]=i
    nx3output[c,1]=rawData[i][j]
    nx3output[c,2]=saliency[i][j]
    j=j+1
    c=c+1
  i=i+1


In [ ]:
import pickle
f = open("/content/drive/MyDrive/BBC Dataset/CNN Summarisation/sentencesToSaliency.pickle", "wb")
pickle.dump(nx3output, f)

In [ ]:
f.close()

In [ ]:
t.to_csv('/content/drive/MyDrive/BBC Dataset/CNN Summarisation/CNNSummariestrain')

Embedding Sentences ith Word2Vec

In [10]:
!cd word2vec/
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

/bin/bash: line 0: cd: word2vec/: No such file or directory
--2021-05-07 11:34:49--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.130.184
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.130.184|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [17]:
import pickle

In [18]:
def loadFromPickle(fileName):
    f = open(fileName, "rb")
    data = pickle.load(f)
    f.close()
    return data 

In [19]:
data = loadFromPickle("/content/drive/MyDrive/BBC Dataset/CNN Summarisation/sentencesToSaliency.pickle")

In [20]:
import gensim
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
#def embed_sentences(data, word2vec_limit = 50000 , NUM_WORDS=20000):

In [21]:
def embed_sentences(data, word2vec_limit = 50000 , NUM_WORDS=20000):   
    '''
    Embed sentences
    Params:
        data             - np.array  [ doc id, sentences, saliency score ]
                            
        word2vec_limit   - int: number of words used in the word embedding provided by Google
                            - ex: 50000
        NUM_WORDS        - int: The maximum number of words to keep, based on word frequency. Only the most common num_words words will be kept.
                            - ex: 20000
       
    Returns:
        input_output        - np.array [embedding matrix , saliency score]
                        
    '''
    i=0
    k=1
    while i<2500:
      #setences ex: ["It's the first sentence!","it is the second sentence"]
      sentences = data[:,1]
      
      #Load Google pre-trained words as a model
      embedding_model = gensim.models.KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=word2vec_limit)
      #Convert the model as a dictionnary word_vectors["hello"] will return a vector like [0.3, 3, ... , -4]
      word_vectors = embedding_model
      #print("Embedding for 'hello': ", word_vectors["hello"], "\n")
      
      # Tokenize the sentences, that is to say convert the 2 sentences ["It's the first sentence!","It is the second sentence"] to 2 sequences [[1 4 2 5 3],[1 6 2 7 3]]
      # It handles the ennoying cases (punctuation, Upper cases, etc...)
      tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',lower=True)
      tokenizer.fit_on_texts(sentences)
      sequences = tokenizer.texts_to_sequences(sentences)
      padded_sequences = pad_sequences(sequences)
      #print("Padded_sequences: ", padded_sequences, "\n")   
      
      #Produce a dictionnary mapping words to tokens e.g. {'it': 1, 'the': 2, 'sentence': 3, 's': 4, 'first': 5, 'is': 6, 'second': 7}
      word_index = tokenizer.word_index
      #print("word_index: " , word_index , "\n" )
      
      #Build a dictionnary mapping tokens to vectors e.g. {'1': [2, ... , -3] ; '2': [4, ... , 0.8] ; ... }
      embedding_weights = {key: embedding_model[word] if word in word_vectors.vocab else
                                np.random.uniform(-0.25, 0.25, word_vectors.vector_size)
                          for word, key in word_index.items()}
      # Add the token "0", used for padding
      embedding_weights[0] = np.zeros(word_vectors.vector_size)
    
      #print("Embedding weights: " , embedding_weights , "\n")
      
      #Build a 3D array: 1D for the sentences, 1D for the words and 1D for the word2vec dimensions.
      print('hey1')
      del sequences
      del embedding_model
      p= padded_sequences[i:i+500]
      del padded_sequences
      print('hey2')
      embedded_sentences = np.stack([np.stack([embedding_weights[token] for token in sentence]) for sentence in p])
      print('hey3')
      del p
      del embedding_weights
      i=i+500
      input_output = np.array([])
      j=i-500
      while j<i:
        input_output = np.append(input_output,np.array([ embedded_sentences[j-(i-500)] , data[j,2] ]) )
        j=j+1
      f = open('/content/drive/MyDrive/BBC Dataset/CNN Summarisation/embeddingsToSaliency'+str(k)+'.pickle', "wb")
      pickle.dump(input_output,f)
      
      k=k+1
      
      del embedded_sentences
      del input_output

    
      #Add back the saliency scores
      #input_output = np.column_stack((embedded_sentences,data[:,2]))    
      
      #input_output = np.array([])
      #for i in range(len(data)):
      #   input_output = np.append(input_output,np.array([ embedded_sentences[i] , data[i,2] ]) )
          
      
      
      #return input_output

In [22]:
embed_sentences(data)

hey1
hey2
hey3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


hey1
hey2
hey3
hey1
hey2
hey3
hey1
hey2
hey3
hey1
hey2
hey3


CNN Model Bilding and training using Keras.Training is done in stages due to large size of Word2Vec  embeddings of data

In [1]:
import pickle
import numpy as np

In [2]:
data = pickle.load(open("/content/drive/MyDrive/BBC Dataset/CNN Summarisation/embeddingsToSaliency1.pickle", "rb"))
#data2 = pickle.load(open("/content/drive/MyDrive/BBC Dataset/CNN Summarisation/embeddingsToSaliency2.pickle", "rb"))


In [5]:
#data = np.concatenate((pickle.load(open("/content/drive/MyDrive/BBC Dataset/CNN Summarisation/embeddingsToSaliency1.pickle", "rb")),pickle.load(open("/content/drive/MyDrive/BBC Dataset/CNN Summarisation/embeddingsToSaliency2.pickle", "rb"))), axis=0)

In [4]:
x = data[::2]
y = data[1::2]
del data

In [5]:
from sklearn.utils import shuffle
x = np.dstack(x)
x = np.rollaxis(x, -1)
x = np.expand_dims(x, axis=1)

mask = y==-1

print("removing -1s...")
x = x[~mask, :]
y = y[~mask]

print("data loaded.")
x, y = shuffle(x, y)

removing -1s...
data loaded.


In [6]:
conv_window_size = (1, 300)
num_filters = 100
reg = 0.01
dropout = 0.5

epochs = 10
batch_size = 16
val_train_ratio = 0.2

In [30]:
!pip install tf-nightly

     |████████████████████████████████| 456.7MB 34kB/s 
     |████████████████████████████████| 4.2MB 45.5MB/s 
     |████████████████████████████████| 471kB 33.7MB/s 
     |████████████████████████████████| 4.0MB 40.7MB/s 
     |████████████████████████████████| 1.3MB 39.1MB/s 
     |████████████████████████████████| 5.9MB 48.1MB/s 
     |████████████████████████████████| 4.9MB 42.7MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.37.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: h5py 2.10.0
    Uninstalli

In [7]:
#import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras import regularizers
from keras.optimizers import Adadelta
def build_model(input_shape, conv_window_size, num_filters, reg, dropout):
    model = Sequential()
    #model.add(Embedding(max_features,300))

    # we add a Convolution 1D, which will learn num_filters
    # word group filters of size conv_window_size:
    model.add(Conv2D(input_shape=input_shape,
                        filters=num_filters,
                        kernel_size=conv_window_size,
                        padding="same",
                        activation="relu",
                        strides=1,
                        ))
    
    model.add(MaxPooling2D(pool_size=(num_filters, 1) ,padding="same")  ) 
    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='softmax', kernel_regularizer=regularizers.l2(reg)))
    
    #In addition, an l2−norm constraint of the weights w_r is imposed during training as well

    model.compile(loss='binary_crossentropy',
                  optimizer=Adadelta(),
                  metrics=['mae'])
    return model

In [8]:
def train(model, x_train, y_train, val_train_ratio=0.2, epochs=1000, batch_size=128):
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_split=val_train_ratio,
                        shuffle=False,
                        verbose=1)
    return history

In [9]:
from keras import backend as K
x = K.cast_to_floatx(x)
y = K.cast_to_floatx(y)

In [10]:
print(x.shape)

(500, 1, 2134, 300)


In [11]:
import tensorflow as tf
#images_nhwc = tf.placeholder(tf.float32, [None, 200, 300, 3])  # input batch
out = tf.transpose(x, [0, 2, 3, 1])
print(out.get_shape())
x=out
del out  # the shape of out is [None, 3, 200, 300]

(500, 2134, 300, 1)


In [ ]:
model = build_model(( x.shape[1], x.shape[2],1), conv_window_size, num_filters, reg, dropout)

history = train(model, x, y, val_train_ratio, epochs, batch_size)

Epoch 1/10
